In [42]:
# Import libraries section

import datetime 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2

from matplotlib.pyplot import rcParams
from scipy import stats

In [5]:
# Some parameters

# Size of the plots
rcParams['figure.figsize'] = 20,6

### Sensitive parameters ###

# Copy them from env file.

In [6]:
# Connection settings

connection = psycopg2.connect(user=db_user,
                                    password=db_password,
                                    host=db_host,
                                    port=db_port,
                                    database=db_name)

In [7]:
df = pd.read_sql("SELECT * FROM product_raw_info", con=connection)

connection.close()

In [8]:
df

,product_name,market_id,unit_scale,source_id,currency_code,date_price,retail_observed_price,wholesale_observed_price
0,Dry Maize,Kiboga : UGA,kg,4,UGX,2019-02-11,600.0,550.0
1,Dry Maize,Kolwezi : DRC,1Kg,1,KES,2019-02-11,61.0,57.0
2,Dry Maize,Nakuru : KEN,1Kg,1,KES,2019-02-11,30.0,18.0
3,Dry Maize,Bujumbura : BDI,1Kg,1,KES,2019-02-11,51.0,48.0
4,Dry Maize,Busia : UGA,1Kg,1,KES,2019-02-10,24.0,20.0
...,...,...,...,...,...,...,...,...
108977,Wood Charcoal,Butare : RWA,1Kg,5,RWF,2019-10-17,8750.0,0.0
108978,Wood Charcoal,Kamembe : RWA,1Kg,5,RWF,2019-10-17,14000.0,0.0
108979,Wood Charcoal,Rushashi : RWA,1Kg,5,RWF,2019-10-17,7500.0,0.0
108980,Wood Charcoal,Nyagatare : RWA,1Kg,5,RWF,2019-10-17,12000.0,0.0


In [9]:
# I'll make a dictionary, grouping all the info available for a product into a key.

# So, what products do we have?

products = list(set(df['product_name']))

print(len(products),'different products are there.')

print('These are them:', products)

176 different products are there.
These are them: ['Cow Peas', 'Chemical Fertilizer', 'Sunflower', 'Imported Rice', 'Exotic Chicken', 'Pearl Millet', 'Fresh Milk', 'Mutton', 'Goat Meat', 'Maize Bran', 'Cow', 'Sweet Potatoes', 'White Fleshed Sweet Potatoes', 'Maize', 'Pineapples', 'Sunflower Seed Cake', 'Finger Millet', 'Exotic Eggs', 'Kayiso Rice', 'Mangoes Local', 'Chic Pea', 'Processed Honey', 'Sun Dried Cassava', 'Green Gram', 'Red Beans', 'Cucumber', 'HORTICULTURE', 'Urea Fertilisers', 'Rice', 'Sunflower Seed Meal', 'Beef', 'Upland Rice', 'Red Irish Potatoes', 'Dolichos (Njahi)', 'Cooking Bananas', 'Maize Meal', 'Cauliflower', 'CEREAL', 'Dry Maize', 'Cow hide', 'NPK Fertilizer', 'Ripe Bananas', 'Nile Perch', 'Groundnuts', 'OTHERS', 'Pigeon Peas', 'Unprocessed Tea', 'Ditane', 'Strawberry', 'White Onions', 'ROOTS & TUBERS', 'Tilapia', 'White Beans', 'Green Maize', 'Goat skin and hide', 'Sorghum Grain', 'Banana Bunch', 'Cabbages', 'Celery', 'Irish Potatoes', 'Beans', 'Eggs', 'Limes', 

In [10]:
# pctwo meaning possible_candidates_to_work_on  
pctwo = []
count = 1
prod_dict = {product:np.nan for product in products}
for product in products:
    available_markets = list(set(df[df['product_name'] == product]['market_id']))
    prod_dict[product] = {market:np.nan for market in available_markets}
    for market in available_markets:
        available_sources = list(set(df[(df['product_name'] == product) & (df['market_id'] == market)]['source_id']))
        prod_dict[product][market] = {source:np.nan for source in available_sources}
        for source in available_sources:
            available_currencies = list(set(df[(df['product_name'] == product) & (df['market_id'] == market) & (df['source_id'] == source)]['currency_code']))
            prod_dict[product][market][source] = {currency:np.nan for currency in available_currencies}
            for currency in available_currencies:
                prod_dict[product][market][source][currency] = {'retail_observed_price':np.nan, 'wholesale_observed_price':np.nan}
                prod_dict[product][market][source][currency]['retail_observed_price'] = {'shape':np.nan, 'info':np.nan}
                prod_dict[product][market][source][currency]['wholesale_observed_price'] = {'shape':np.nan, 'info':np.nan}
                
                prod_dict[product][market][source][currency]['retail_observed_price']['shape'] = df[(df['product_name'] == product) & (df['market_id'] == market) & (df['source_id'] == source) & (df['currency_code'] == currency)][['date_price','unit_scale','retail_observed_price']].shape
                prod_dict[product][market][source][currency]['retail_observed_price']['info'] = df[(df['product_name'] == product) & (df['market_id'] == market) & (df['source_id'] == source) & (df['currency_code'] == currency)][['date_price','unit_scale','retail_observed_price']]
                if (prod_dict[product][market][source][currency]['retail_observed_price']['shape'][0] > 156) & (prod_dict[product][market][source][currency]['retail_observed_price']['info']['date_price'].max() > datetime.date(2019, 1, 1)):
                    pctwo.append(('product_'+ str(count), product, market, source, currency,'retail'))
                    count +=1
                
                prod_dict[product][market][source][currency]['wholesale_observed_price']['shape'] = df[(df['product_name'] == product) & (df['market_id'] == market) & (df['source_id'] == source) & (df['currency_code'] == currency)][['date_price','unit_scale','wholesale_observed_price']].shape
                prod_dict[product][market][source][currency]['wholesale_observed_price']['info'] = df[(df['product_name'] == product) & (df['market_id'] == market) & (df['source_id'] == source) & (df['currency_code'] == currency)][['date_price','unit_scale','wholesale_observed_price']]
                if (prod_dict[product][market][source][currency]['wholesale_observed_price']['shape'][0] > 156) & (prod_dict[product][market][source][currency]['wholesale_observed_price']['info']['date_price'].max() > datetime.date(2019, 1, 1)):
                    pctwo.append(('product_'+ str(count), product, market, source, currency,'wholesale'))
                    count +=1
            
print(len(pctwo), 'possible products/markets we can work on.')
pctwo

84 possible products/markets we can work on.


[('product_1', 'Imported Rice', 'Gitega : BDI', 1, 'KES', 'retail'),
 ('product_2', 'Imported Rice', 'Gitega : BDI', 1, 'KES', 'wholesale'),
 ('product_3', 'Imported Rice', 'Kampala : UGA', 1, 'KES', 'retail'),
 ('product_4', 'Imported Rice', 'Kampala : UGA', 1, 'KES', 'wholesale'),
 ('product_5', 'Green Gram', 'Nairobi : KEN', 1, 'KES', 'retail'),
 ('product_6', 'Green Gram', 'Nairobi : KEN', 1, 'KES', 'wholesale'),
 ('product_7', 'Green Gram', 'Lira : UGA', 1, 'KES', 'retail'),
 ('product_8', 'Green Gram', 'Lira : UGA', 1, 'KES', 'wholesale'),
 ('product_9', 'Green Gram', 'Mombasa : KEN', 1, 'KES', 'retail'),
 ('product_10', 'Green Gram', 'Mombasa : KEN', 1, 'KES', 'wholesale'),
 ('product_11', 'Rice', 'Gitega : BDI', 1, 'KES', 'retail'),
 ('product_12', 'Rice', 'Gitega : BDI', 1, 'KES', 'wholesale'),
 ('product_13', 'Dry Maize', 'Kimironko : RWA', 1, 'KES', 'retail'),
 ('product_14', 'Dry Maize', 'Kimironko : RWA', 1, 'KES', 'wholesale'),
 ('product_15', 'Dry Maize', 'Gitega : BDI',

In [122]:
# pctwo = possible_candidates_to_work_on

In [126]:
pctwo[0][0]

'product_1'

In [11]:
conditional = (df['product_name'] == pctwo[0][1]) & (df['market_id'] == pctwo[0][2]) & (df['source_id'] == pctwo[0][3]) & (df['currency_code'] == pctwo[0][4])
df[conditional][['date_price','unit_scale','wholesale_observed_price']]


,date_price,unit_scale,wholesale_observed_price
63646,2019-02-12,1Kg,136.0
63654,2019-02-13,1Kg,135.0
63661,2019-02-14,1Kg,135.0
63665,2019-02-15,1Kg,135.0
63682,2019-02-18,1Kg,135.0
...,...,...,...
65762,2019-10-07,1Kg,135.0
65785,2019-10-09,1Kg,135.0
65788,2019-10-11,1Kg,135.0
65804,2019-10-15,1Kg,135.0


In [159]:
def clean_df(dataframe):
    
    ''' 
    Removes duplicates in dates column. 
    Try to correct typos.
    Verify unique unit scale.
    
    Make a dataframe with the last Sunday before the dates of the input dataframe, and the saturday of the last week in within the dates.
    Then Merge both dataframes to have one with all the possible weeks within the dates of the original dataframe.
    '''
    
    cfd = dataframe.copy()
    
    initial_shape = dataframe.shape
    
    # Reset index
    
    cfd = cfd.reset_index(drop=True)
    
    # Remove duplicates in dates column.
    
    drop_index = list(cfd[cfd.duplicated(['date_price'], keep='first')].index)
    
    cfd = cfd.drop(labels=drop_index, axis=0).reset_index(drop=True)
    
    # Drop outliers - the first round will face typos, the seconds truly outliers.
    
    z = np.abs(stats.zscore(cfd.iloc[:,-1], nan_policy='omit'))
    
    drop_index = list(np.where(z>4)[0])   
   
    cfd = cfd.drop(labels=drop_index, axis=0).reset_index(drop=True)
    
    # Second round.
    
    z = np.abs(stats.zscore(cfd.iloc[:,-1], nan_policy='omit'))
    
    drop_index = list(np.where(z>5)[0])
    
    cfd = cfd.drop(labels=drop_index, axis=0).reset_index(drop=True)
    
    # Consider the mode of unit scale as the one.
    
    mode = stats.mode(cfd.iloc[:,1])[0][0]
    
    discording_scale = list(cfd[cfd['unit_scale'] != mode].index)
    
    if discording_scale:
        
        cfd = cfd.drop(labels=discording_scale, axis=0).reset_index(drop=True)
    
    # Turn the dataframe into a calendar.
    
    if datetime.date.weekday(cfd['date_price'].min()) == 6:
        start = datetime.date.weekday(cfd['date_price'].min())
    else:
        start = cfd['date_price'].min() - datetime.timedelta(days=datetime.date.weekday(cfd['date_price'].min()) + 1)
    if datetime.date.weekday(cfd['date_price'].max()) == 5:
        end = datetime.date.weekday(cfd['date_price'].min())
    else:
        end = cfd['date_price'].max() + datetime.timedelta(days=(5 - datetime.date.weekday(cfd['date_price'].max())))
        
    dummy = pd.DataFrame()
    dummy['date_price'] = pd.date_range(start=start, end=end)
    dummy = dummy.set_index('date_price')
    cfd = cfd.set_index('date_price')
    cfd = dummy.merge(cfd,how='outer',left_index=True, right_index=True)
    del dummy

    cfd[['unit_scale']] = cfd[['unit_scale']].fillna(mode)
    
    final_shape = cfd.shape
    
    na_percentage = round(cfd.iloc[:,-1].isnull().sum() / final_shape[0],6)
    
    
    # Consider only workdays
    
    cfd_wd = cfd[cfd.index.dayofweek < 5]

    final_wd_shape = cfd_wd.shape
    
    na_wd_percentage = round(cfd_wd.iloc[:,-1].isnull().sum() / final_wd_shape[0],6)
    
    return ((cfd,initial_shape,final_shape,na_percentage),(cfd_wd,initial_shape,final_wd_shape,na_wd_percentage))


In [155]:
clean_df(df[conditional][['date_price','unit_scale','wholesale_observed_price']])

(0.537267, 0.352174)

In [166]:
### Wholesale


for i in range(len(pctwo)):
    
    conditional = (df['product_name'] == pctwo[i][1]) & (df['market_id'] == pctwo[i][2]) & (df['source_id'] == pctwo[i][3]) & (df['currency_code'] == pctwo[i][4])
    print(i, 'product:',pctwo[i][1], ' // market:', pctwo[i][2], ' // source:', pctwo[i][3],  ' // currency:', pctwo[i][4], '// ', (clean_df(df[conditional][['date_price','unit_scale','wholesale_observed_price']])[0][3], clean_df(df[conditional][['date_price','unit_scale','wholesale_observed_price']])[1][3]))




0 product: Imported Rice  // market: Gitega : BDI  // source: 1  // currency: KES //  (0.684932, 0.558904)
1 product: Imported Rice  // market: Gitega : BDI  // source: 1  // currency: KES //  (0.684932, 0.558904)
2 product: Imported Rice  // market: Kampala : UGA  // source: 1  // currency: KES //  (0.691837, 0.568571)
3 product: Imported Rice  // market: Kampala : UGA  // source: 1  // currency: KES //  (0.691837, 0.568571)
4 product: Green Gram  // market: Nairobi : KEN  // source: 1  // currency: KES //  (0.700952, 0.581333)
5 product: Green Gram  // market: Nairobi : KEN  // source: 1  // currency: KES //  (0.700952, 0.581333)
6 product: Green Gram  // market: Lira : UGA  // source: 1  // currency: KES //  (0.710476, 0.594667)
7 product: Green Gram  // market: Lira : UGA  // source: 1  // currency: KES //  (0.710476, 0.594667)
8 product: Green Gram  // market: Mombasa : KEN  // source: 1  // currency: KES //  (0.698842, 0.578378)
9 product: Green Gram  // market: Mombasa : KEN  //

In [ ]:
# conditional = (df['product_name'] == pctwo[16][1]) & (df['market_id'] == pctwo[16][2]) & (df['source_id'] == pctwo[16][3]) & (df['currency_code'] == pctwo[16][4])
# df[conditional][['date_price','unit_scale','wholesale_observed_price']].reset_index(drop=True).iloc[130:140,:]
# clean_df(df[conditional][['date_price','unit_scale','wholesale_observed_price']])

In [168]:
### Retail


for i in range(len(pctwo)):
    
    conditional = (df['product_name'] == pctwo[i][1]) & (df['market_id'] == pctwo[i][2]) & (df['source_id'] == pctwo[i][3]) & (df['currency_code'] == pctwo[i][4])
    print(i, 'product:',pctwo[i][1], ' // market:', pctwo[i][2], ' // source:', pctwo[i][3],  ' // currency:', pctwo[i][4], '// ', (clean_df(df[conditional][['date_price','unit_scale','retail_observed_price']])[0][3], clean_df(df[conditional][['date_price','unit_scale','retail_observed_price']])[1][3]))




0 product: Imported Rice  // market: Gitega : BDI  // source: 1  // currency: KES //  (0.684932, 0.558904)
1 product: Imported Rice  // market: Gitega : BDI  // source: 1  // currency: KES //  (0.684932, 0.558904)
2 product: Imported Rice  // market: Kampala : UGA  // source: 1  // currency: KES //  (0.691837, 0.568571)
3 product: Imported Rice  // market: Kampala : UGA  // source: 1  // currency: KES //  (0.691837, 0.568571)
4 product: Green Gram  // market: Nairobi : KEN  // source: 1  // currency: KES //  (0.700952, 0.581333)
5 product: Green Gram  // market: Nairobi : KEN  // source: 1  // currency: KES //  (0.700952, 0.581333)
6 product: Green Gram  // market: Lira : UGA  // source: 1  // currency: KES //  (0.710476, 0.594667)
7 product: Green Gram  // market: Lira : UGA  // source: 1  // currency: KES //  (0.710476, 0.594667)
8 product: Green Gram  // market: Mombasa : KEN  // source: 1  // currency: KES //  (0.698842, 0.578378)
9 product: Green Gram  // market: Mombasa : KEN  //